In [3]:
import requests
from geopy.distance import geodesic

def get_nearest_subway_station(lat, lon, kakao_api_key):
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}
    
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    params = {
        "query": "지하철역",
        "x": lon,
        "y": lat,
        "radius": 2000,
        "sort": "distance"
    }
    
    response = requests.get(url, headers=headers, params=params)
    result = response.json()
    if not result['documents']:
        return None, None, None
    
    nearest = result['documents'][0]
    station_name = nearest['place_name']
    station_lat = float(nearest['y'])
    station_lon = float(nearest['x'])

    distance = geodesic((lat, lon), (station_lat, station_lon)).meters
    return station_name, (station_lat, station_lon), distance

kakao_api_key = "f32e9311c06e941799af2d38345741de"
lat, lon = 37.4456097972955,126.893893353662
station, coords, dist = get_nearest_subway_station(lat, lon, kakao_api_key)
print(f"가장 가까운 역: {station}, 거리: {dist:.1f}m")

가장 가까운 역: 시흥사거리역 (2026년 12월 예정), 거리: 1119.4m


In [7]:
import pandas as pd
import requests
from geopy.distance import geodesic
import time

def get_nearest_subway_station(lat, lon, kakao_api_key):
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    params = {
        "query": "지하철역",
        "x": lon,
        "y": lat,
        "radius": 2000,
        "sort": "distance"
    }
    try:
        response = requests.get(url, headers=headers, params=params)
        result = response.json()
        # documents 키가 없으면 에러 메시지 출력
        if 'documents' not in result:
            print(f"API Error: {result}")
            return None, None
        if not result['documents']:
            return None, None
        nearest = result['documents'][0]
        station_name = nearest['place_name']
        station_lat = float(nearest['y'])
        station_lon = float(nearest['x'])
        distance = geodesic((lat, lon), (station_lat, station_lon)).meters
        return station_name, distance
    except Exception as e:
        print(f"Exception: {e}")
        return None, None

df = pd.read_csv('../KIC/KIC_Cord.csv', header=None, names=['name', 'lat', 'lon'])

kakao_api_key = "f32e9311c06e941799af2d38345741de"

results = []

for idx, row in df.iterrows():
    lat, lon = row['lat'], row['lon']
    station, dist = get_nearest_subway_station(lat, lon, kakao_api_key)
    results.append([row['name'], lat, lon, station, dist])
    time.sleep(0.2)

result_df = pd.DataFrame(results, columns=['name', 'lat', 'lon', 'nearest_station', 'distance'])

result_df.to_csv('KIC-Subway.csv', index=False, encoding='utf-8-sig')

API Error: {'code': -2, 'msg': 'Invalid Parameter (x)'}
Exception: Point coordinates must be finite. (1.0, nan, 0.0) has been passed as coordinates.
